<a href="https://colab.research.google.com/github/felipeabe/artificial-neural-network/blob/projetoComCSV/trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#uploading files
from google.colab import files
uploaded = files.upload()

#importing libs
import pandas as pd

Saving alzheimer.csv to alzheimer.csv


In [2]:
dfb = pd.read_csv('alzheimer.csv')

# Selecionar colunas úteis
columns_to_keep = [
    'Age', 'BMI', 'AlcoholConsumption', 'PhysicalActivity',
    'DietQuality', 'SleepQuality', 'Smoking',
    'FamilyHistoryAlzheimers', 'Depression', 'Hypertension', 'Diagnosis'
]

# Filtrar o DataFrame
filtered_alzheimer_data = dfb[columns_to_keep]

# Verificar valores ausentes
missing_values = filtered_alzheimer_data.isnull().sum()

# Preencher valores ausentes (se houver)
filtered_alzheimer_data.fillna(filtered_alzheimer_data.median(), inplace=True)

# Salvar em um novo CSV
cleaned_file_path = 'alzheimer_cleaned.csv'
filtered_alzheimer_data.to_csv(cleaned_file_path, index=False)

missing_values, cleaned_file_path


<ipython-input-2-cd5cf057ebfb>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_alzheimer_data.fillna(filtered_alzheimer_data.median(), inplace=True)


(Age                        0
 BMI                        0
 AlcoholConsumption         0
 PhysicalActivity           0
 DietQuality                0
 SleepQuality               0
 Smoking                    0
 FamilyHistoryAlzheimers    0
 Depression                 0
 Hypertension               0
 Diagnosis                  0
 dtype: int64,
 'alzheimer_cleaned.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separar as variáveis explicativas e alvo
X = filtered_alzheimer_data.drop(columns=['Diagnosis'])
y = filtered_alzheimer_data['Diagnosis']

# Tratar valores faltantes em X
X = X.fillna(X.mean())

# Normalizar os dados (apenas X)
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Dividir os dados em treino e teste (stratified split para balancear classes)
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42, stratify=y
)

# Redefinir os índices para alinhamento
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Combinar novamente X e y para salvar como arquivos separados
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# Salvar os arquivos
train_file_path = 'alzheimer_train.csv'
test_file_path = 'alzheimer_test.csv'

train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)

# Checar dados salvos
print("Primeiras linhas do conjunto de treino:")
print(train_data.head())

print("Primeiras linhas do conjunto de teste:")
print(test_data.head())


Primeiras linhas do conjunto de treino:
        Age       BMI  AlcoholConsumption  PhysicalActivity  DietQuality  \
0  0.300000  0.747718            0.338947          0.330269     0.377028   
1  0.866667  0.163598            0.936547          0.881996     0.036663   
2  0.866667  0.526223            0.931950          0.654257     0.130851   
3  0.066667  0.996893            0.871020          0.661547     0.967679   
4  0.300000  0.399295            0.932860          0.121732     0.465150   

   SleepQuality  Smoking  FamilyHistoryAlzheimers  Depression  Hypertension  \
0      0.496147      1.0                      0.0         0.0           0.0   
1      0.411456      0.0                      1.0         0.0           0.0   
2      0.936762      0.0                      1.0         0.0           0.0   
3      0.857746      0.0                      0.0         0.0           0.0   
4      0.661064      0.0                      0.0         0.0           0.0   

   Diagnosis  
0          0 

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Funções de ativação e derivadas
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

# Função de perda (Binary Cross-Entropy)
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Classe para a Rede Neural
class NeuralNetwork:
    def __init__(self, layers, activations, learning_rate=0.01, loss_function='binary_cross_entropy'):
        assert len(layers) - 1 == len(activations), \
            "Número de funções de ativação deve ser igual ao número de camadas ocultas + saída."
        self.layers = layers
        self.learning_rate = learning_rate
        self.activations = activations
        self.loss_function_name = loss_function
        self.activation_functions = {
            'sigmoid': (sigmoid, sigmoid_derivative),
            'relu': (relu, relu_derivative),
            'tanh': (tanh, tanh_derivative)
        }
        self.loss_functions = {
            'binary_cross_entropy': binary_cross_entropy
        }
        self.loss_function = self.loss_functions[loss_function]

        # Inicialização dos pesos e bias
        self.weights = [np.random.randn(layers[i], layers[i - 1]) * 0.01 for i in range(1, len(layers))]
        self.biases = [np.zeros((layers[i], 1)) for i in range(1, len(layers))]

    def forward(self, X):
        self.a = [X.T]  # Ativações (saídas) por camada
        self.z = []     # Soma ponderada (z = W.a + b)

        for i in range(len(self.weights)):
            z = np.dot(self.weights[i], self.a[-1]) + self.biases[i]
            self.z.append(z)
            activation_function = self.activation_functions[self.activations[i]][0]
            self.a.append(activation_function(z))

        return self.a[-1].T  # Saída final da rede

    def backward(self, X, y):
        m = X.shape[0]
        y = y.reshape(-1, 1).T
        d_weights = [np.zeros_like(w) for w in self.weights]
        d_biases = [np.zeros_like(b) for b in self.biases]

        # Cálculo do erro na saída
        dz = self.a[-1] - y  # Erro na saída
        for i in reversed(range(len(self.weights))):
            dw = np.dot(dz, self.a[i].T) / m
            db = np.sum(dz, axis=1, keepdims=True) / m
            d_weights[i] = dw
            d_biases[i] = db

            if i > 0:  # Propagação do erro para trás
                activation_derivative = self.activation_functions[self.activations[i - 1]][1]
                dz = np.dot(self.weights[i].T, dz) * activation_derivative(self.z[i-1])

        # Atualização dos pesos e bias
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * d_weights[i]
            self.biases[i] -= self.learning_rate * d_biases[i]

    def train(self, X, y, epochs=100):
        print("Treinamento iniciado.")
        print(f"Funções de ativação usadas por camada: {self.activations}")
        print(f"Função de perda: {self.loss_function_name}")

        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.loss_function(y, y_pred)
            self.backward(X, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")
        print("Treinamento concluído.")

    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)

# Dados de treino e teste
train_data = pd.read_csv('alzheimer_train.csv')
test_data = pd.read_csv('alzheimer_test.csv')

X_train = train_data.drop(columns=['Diagnosis']).values
y_train = train_data['Diagnosis'].values
X_test = test_data.drop(columns=['Diagnosis']).values
y_test = test_data['Diagnosis'].values

# Criar e treinar o modelo
nn = NeuralNetwork(
    layers=[X_train.shape[1], 10, 5, 1],
    activations=['relu', 'tanh', 'sigmoid'],
    learning_rate=0.01,
    loss_function='binary_cross_entropy'
)
nn.train(X_train, y_train, epochs=100)

# Avaliar o modelo
y_pred = nn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on Test Data: {accuracy:.2%}")


Treinamento iniciado.
Funções de ativação usadas por camada: ['relu', 'tanh', 'sigmoid']
Função de perda: binary_cross_entropy
Epoch 1/100, Loss: 0.6931
Epoch 2/100, Loss: 0.6929
Epoch 3/100, Loss: 0.6927
Epoch 4/100, Loss: 0.6925
Epoch 5/100, Loss: 0.6923
Epoch 6/100, Loss: 0.6921
Epoch 7/100, Loss: 0.6919
Epoch 8/100, Loss: 0.6917
Epoch 9/100, Loss: 0.6915
Epoch 10/100, Loss: 0.6913
Epoch 11/100, Loss: 0.6911
Epoch 12/100, Loss: 0.6909
Epoch 13/100, Loss: 0.6906
Epoch 14/100, Loss: 0.6904
Epoch 15/100, Loss: 0.6902
Epoch 16/100, Loss: 0.6900
Epoch 17/100, Loss: 0.6898
Epoch 18/100, Loss: 0.6897
Epoch 19/100, Loss: 0.6895
Epoch 20/100, Loss: 0.6893
Epoch 21/100, Loss: 0.6891
Epoch 22/100, Loss: 0.6889
Epoch 23/100, Loss: 0.6887
Epoch 24/100, Loss: 0.6885
Epoch 25/100, Loss: 0.6883
Epoch 26/100, Loss: 0.6881
Epoch 27/100, Loss: 0.6879
Epoch 28/100, Loss: 0.6877
Epoch 29/100, Loss: 0.6875
Epoch 30/100, Loss: 0.6874
Epoch 31/100, Loss: 0.6872
Epoch 32/100, Loss: 0.6870
Epoch 33/100, Loss